In [1]:

from scipy.integrate import ode

%pylab inline

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
matplotlib.use("Agg")
import scipy.io as sio
import scipy.signal
from pylab import *

asin=np.arcsin
sqrt=np.sqrt
sin=np.sin
cos=np.cos
pi=np.pi
exp=np.exp
tan=np.tan
I=1j
inf=np.inf

hilbert=scipy.signal.hilbert


Populating the interactive namespace from numpy and matplotlib


/usr/lib/pymodules/python2.7/matplotlib/__init__.py:1173: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [ ]:
# Here we code up the Daines et al model for subcellular resource allocation.
# A plankton will be parametrized by two parameters, the cell radius and the 
# allocation to light harvesting machinery. The percent mass of a cell will be decided as
# S+R+L=1.0.


# First we write the membrane volume, which is a function of the membrane thickness and periplasmic thickness:
# To do this we need some constants. Our units will be in nanometers

tcm = 9.0 # cytoplasmic membrane width
tom = 9.0 # outer membrane width
tp = 15.0 # periplasm width
# The units of volume with be cubic micrometers, which are cubic nm times 10**(-9)
def Vmem(r):
    Vom = 4.0*pi/3.0*(r**3-(r-tom)**3)
    Vcm = 4.0*pi/3.0*((r-tom-tp)**3-(r-tom-tp-tcm)**3)
    return (Vom+Vcm)*(1.0e-9)

# Here we have reference values for the density of a membrane and the volume of a membrane:
rhoMemStandard = 1.15 # In units of Mg/m^(-3)
VMemStandard = .19 # In units of micrometers ^3

# Now we can define the mass of the membrane, both as a function of the dimensional and relative density and volume

def massMem(rhoMem,VMem):
    return rhoMem*VMem # This is in units of femtograms

def massMemND(rhoMemRel,VMemRel):
    return 220*rhoMemRel*VMemRel # In units of femtograms

# Now we calculate the overall cellular dry mass. Here pDryWeight is a percent dry weight, rho is density in Mg/m^3 and
# r is in units of micrometers

def mDryCell(pDryWeight,rho,r):
    return 4*pi/3.0*pDryWeight*rho*r**3*10**(6)*10**(-18) # This weight is in units of grams
    
# S(r) is the percent of the cell dry mass that is associated with structure. It depends on the DNA mass, which could be modelled
# either as a constant or as a function of cell size. All mass will be in femtograms

mDNA = 1.8  
rho=1.0
pDryWeight=.47

alphaS = .12
gamma = .2


def S(r):   
    return .12/r+.2  # This is in units of percent dry weight
    

D = 3.6e7/24 # Diffusion coefficient in micrometers^2/h
DFe = 3.6e7/24.0
# Here are costs of synthesis and maintenance of different cellular components
PhiS = .67
PhiMT0 = .001
T0= 25.0
QPhi = 2.0

def PhiM(T):
    return PhiMT0*QPhi**((T-T0)/10)

# We define the photosynthesis rate

kP=1.15*3.9*10**(-3) # Photosynthetic efficiency in units of the photosynthetic apparatus in units of m^2 s /microEinsteins hours
def fPhoto(L,I):
    return kP*L*I

# Now for the biosynthesis terms
kST0 = .168 # Biosynthesis efficiency at temperature T=T0
Q10k = 2.0
def fBio(E,T):
    return E*kST0*Q10k**((T-T0)/10.0)


# Now for uptake rate terms:

# We begin with P uptake. In order to define P uptake, we need to determine what VMaxP and KP are as functions of radius.
# c1=1e-15*4*pi/3.0/24.0
c1 = 2e-15*4*pi/3.0/24.0 # This is in units of mole P/cell/hr/micrometer^2

# We calculate the affinity at r=1

def Affinity(r):
    return 4*pi*D*r*1e-15

def VMaxP(r):
    return c1*r**2 # Units of micromoles P/cell/hr

c2 = VMaxP(1.0)/Affinity(1.0) # This is in units of micromol P/L/micrometer

def KP(r):
    return c2*r  # This is micromol P/L

# Now we can use monod kinetics

def uptakeRateP(r,P):
    return VMaxP(r)*P/(P+KP(r))  # This is micromol P/hr

# Next we define the corresponding function for Fe


c4 = 100*10**(-3.0) # microMol Fe/L
c3 = 1.5*10**(-8)/24.0 #micromol Fe/hr
def VMaxFe(r):
    return c3*r**2/25  # Units of microMol Fe/hr

def KFe(r):
    return c4*r/5  # units of micromol Fe

def uptakeRateFe(r,Fe):
    #return VMaxFe(r)*Fe/(Fe+KFe(r))
    return DFe*4*pi*r*Fe*1e-15
    


# In order to calculate the growth rate of a strategy, we need to calculate the quota of said strategy


alphaE=.55
PPhosphoLipid = .042
PDNA = .095
PRibosomeEu = .047

NProtein = .16
NRibosomeEu = .156
NDNA = .16
NPhospholipid = .016

CProtein = .53
CDNA = .36
CPhospholipid = .65
CRibosomeEu = .419

CCarb = .4
CLipid = .76

# From this ratio we can calculate the stoichiometry (N:P) of the biosynthesis component. Essentially we use the rule that 
# N is 16% of dry mass, and that P is 4.2% by mass of the phospholipid pool, and that P is 4.7% by mass of the Ribosome pool.

def totalP(E,r):
    return ((.3*alphaS/r+.05)*PPhosphoLipid+alphaE*E*PRibosomeEu+.01*PDNA)/31

def totalN(E,r):
    return ((.3*alphaS/r+.05)*NPhospholipid+.7*alphaS*NProtein/r+(1-alphaE)*E*NProtein+alphaE*E*NRibosomeEu+(1-E-alphaS/r-gamma)*NProtein+.01*NDNA)/14.0

def totalC(E,r):
    return ((.3*alphaS/r+.05)*CPhospholipid+.7*alphaS*CProtein/r+.10*CLipid+.4*CCarb+(1-alphaE)*E*CProtein+alphaE*E*CRibosomeEu+(1-E-alphaS/r-gamma)*CProtein+.01*CDNA)/12.0




FePerL = 5e-6    # mol Fe per gram dry mass of the light gathering machinery




def QFe(r,L):
    return mDryCell(pDryWeight,rho,r)*L*FePerL   #.5/12*38*1e-6*2
def QP(r,L):
    return mDryCell(pDryWeight,rho,r)*totalP(1-S(r)-L,r)

# Now we use the quotas to define the growth rates

def muFe(r,L,Fe):
    return uptakeRateFe(r,Fe)/QFe(r,L)

def muP(r,L,P):
    return uptakeRateP(r,P)/QP(r,L)



#we are in position to define the growth rate as a function of environmental conditions and strategy

def mu(r,E,I,T,P,Fe):
    L = 1-S(r)-E
    #print L
    BioSynthesisRate = min([(fPhoto(1-S(r)-E,I)-PhiM(T))/(1+PhiS), fBio(E,T), muP(r,L,P),muFe(r,L,Fe)])
    if BioSynthesisRate>0:
        return BioSynthesisRate
    return min([0,fPhoto(1-S(r)-E,100)-PhiM(T)])


# In this function we will calculate the optimal strategy 

def optimalStrategy(I,T,P,Fe):
    # The first step will be to find E as a function of r
    CE = alphaE*PRibosomeEu/31.0
    KST = kST0*Q10k**((T-T0)/10.0)
    a=CE*KST  
    CP = .3*alphaS*PPhosphoLipid/31.0
    CS = (.05*PPhosphoLipid + .01*PDNA)/31.0
    def b(r):
        return KST/31.0*(CP/r+CS)
    def c(r):
        return -VMaxP(r)*P/(P+KP(r))*1/(mDryCell(pDryWeight,rho,r))
    
    
    def E(r):
        return (-b(r)+sqrt(b(r)**2-4*a*c(r)))/(2*a)
    
    def zeroFunction(r):
        return fBio(E(r),T)-(fPhoto(1-S(r)-E(r),I)-PhiM(T))/(1+PhiS)    
    
    def rMinFunction(r):
        return muP(r,0,P)-fBio(1-S(r)-E(r),10)
    rMin = scipy.optimize.brentq(rMinFunction,alphaS/(1-gamma),1000)
    rOpt=scipy.optimize.brentq(zeroFunction,alphaS/(1-gamma), 1000)
    
    # Now we find the optimal Fe limited strategy
    aFe = kP*I/(1+PhiS)
    bFe = -PhiM(T)/(1+PhiS)
    def cFe(r):
        return -uptakeRateFe(r,Fe)/(mDryCell(pDryWeight,rho,r)*FePerL)
    def LFe(r):
        return (-bFe+sqrt(bFe**2-4*aFe*cFe(r)))/(2*aFe)
    def zeroFunctionFe(r):
        return muFe(r,LFe(r),Fe)-fBio(1-S(r)-LFe(r),T)
    rOptFe = scipy.optimize.brentq(zeroFunctionFe,.15,1000)
    if mu(rOpt,E(rOpt),I,T,P,Fe)>mu(rOptFe,1-S(rOptFe)-LFe(rOptFe),I,T,P,Fe):
        return [rOpt,E(rOpt)]
    return [rOptFe,1-S(rOptFe)-LFe(rOptFe)]
    

def optimalCP(I,T,P,Fe):
    a1=optimalStrategy(I,T,P,Fe)
    r=a1[0]
    E=a1[1]
    PLimitedCP = totalC(E,r)/totalP(E,r)
    if mu(r,E,I,T,P,Fe)< muP(r,1-S(r)-E,P):
        return PLimitedCP*(mu(r,E,I,T,P,Fe)/muP(r,1-S(r)-E,P))
    return PLimitedCP

In [618]:
# First we define the set of constants that parametrize our model:

T = 25.0e6  # units of m^3/s
fhd = 38.1e6  # units of m^3/s
rpa = 38    # units of equivalents Alk/mol P (not sure what an equivalent is)

def rpch(P):
    return optimalCP(IH,TH,P,FeH)

def rpcl(P):
    return optimalCP(IL,TL,P,FeL)


rpch = 70 # units of mol C/mol P
rpcl = 131
phP = 2.31e9/131.0
ph = phP*rpch 
#pl = 5e5/rac      # units of mol C/S (not sure why it is C and not P)
fah = 3*.15*36.1e12 # units of m^3/day
fal = 3*.85*36.1e12      # units of m^3/day
volumeH = 36e12*.15*250 # units of m^3
volumeL = 36e12*.85*100 # units of m^3
volumeD = 36e12*4000 # units of m^3
molesAtmosphere = 1.8e20 # units of mol Gas


FeH = 1.5e-10
IH = 30
FeL = 5e-9
IL = 100

TH = 2.5   # Temperature in degrees Centigrade in the high lattitude box
TL = 21.0  # Low Lattitude Temperature in degrees C
TD = 2.0   # Temperature ofthe deep ocean in degrees C
SH = 34.70  # High Lattitude Salinity in Practical Salinity Units
SL = 34.70 # Low Lattitude salinity in Practical Salinity Units
SD = 34.70  # Deep Ocean Salinity in Practical Salinity Units
PL = 0 # Low Lattitude PO4 in mumol/L


phP = ph/rpch


def plP(PD):  # milimol P/s
    return T*PD-T*PL


def pl(PD):
    return (T*PD-T*PL)*rpcl

# We are going to define a 4 box model of the atmosphere and oceans,
# The high and low lattitude surface oceans will each have a box, as
# will the global atmosphere and the deep ocean. The dynamical variables
# will be each the concentration of Phosphorous, Dissolved Inorganic 
# Carbon, and alkalinity in the ocean boxes, and pC02 in the atmosphere
# box. 

# We will define the flux of each element individually

# These P fluxes have units of microMol/L water/s * m^3 water = milimol P/s

def fluxPH(PD,PH,PL): 
    if PH < 1e-3:
        return max([0,T*PL-(T+fhd)*PH+fhd*PD-phP])
    return T*PL-(T+fhd)*PH+fhd*PD-phP

def fluxPD(PD,PH,PL):
    
    if PH < 1e-3 and T*PL-(T+fhd)*PH+fhd*PD-phP<0:
        return T*PL-T*PD+plP(PD)
    return (T+fhd)*PH-(T+fhd)*PD+plP(PD)+phP

#def fluxPL(PD,PH,PL):
#    return T*PD-T*PL-pl

# These Alkalinity fluxes have units of miliEquivalents of Alkalinity per second (using similar arguments as to P)


def fluxAH(AD,AH,AL,PD,PH,PL):
    if PH<1e-3 and T*PL-(T+fhd)*PH+fhd*PD-phP<0:
        return T*AL-(T+fhd)*AH+fhd*AD-(T*PL-(T+fhd)*PH+fhd*PD)*rpa
    return T*AL-(T+fhd)*AH+fhd*AD-phP*rpa

def fluxAD(AD,AH,AL,PD,PH,PL):
    if PH<1e-3 and T*PL-(T+fhd)*PH+fhd*PD-phP<0:
        return (T+fhd)*AH+(T*PL-(T+fhd)*PH+fhd*PD)*rpa-(T+fhd)*AD+plP(PD)*rpa
    return (T+fhd)*AH+phP*rpa-(T+fhd)*AD+plP(PD)*rpa

def fluxAL(AD,AH,AL,PD):
    return T*AD-T*AL-plP(PD)*rpa



# These Carbon fluxes have the units milimol C/S. We have to apply an adjustment term to the parts with atmospheric
# flux, which were initially on a per day schedule.

secondsPerDay = 3600.0


def fluxCD(CD,CH,CL,PD, PH, PL):
    if PH<1e-3 and T*PL-(T+fhd)*PH+fhd*PD-phP<0:
        return (T+fhd)*CH-(T+fhd)*CD+(T*PL-(T+fhd)*PH+fhd*PD)/rpch+pl(PD)
    return (T+fhd)*CH-(T+fhd)*CD+ph+pl(PD)

# In order to calculate the flux of carbon into the upper ocean boxes
# and the atmosphere we need to calculate the pCO2 of the upper ocean 
# boxes


# The units of solubility are, for K0, mol/kg/atm, and for K1 and K2 mol/kg

def K0(S,T):
    return exp(-60.2409+93.4517*100/(273+T)+23.3585*log((T+273)/100)+S*(.023517-.023656*(T+273)/100+.0047036*((T+273)/100)**2))

def lnK(S,T):
    return -60.2409+93.4517/(273+T)+23.3585*log((T+273)/100)+S*(.023517-.023656*(T+278)/100+.0047036*((T+273)/100)**2)


def K1(S,T):
    return 1.0e6*exp(log(10)*(62.008-3670.7/(273+T)-9.7944*log(T+273)+.0118*S-.000116*S**2))

def K2(S,T):
    return 1.0e6*exp(-log(10)*(4.777+1394.7/(273+T)-.0184*S+.000118*S**2))


def KW(S,T):
    TK=T+273.0
    return 1.0e12*exp((148.96502-13847.26/TK-23.6521*log(TK)+sqrt(S)*(-5.977+118.67/TK+1.0495*log(TK))-.01615*S))

def KB(S,T):
    TK = T+273.0
    return 1.0e6*exp((1.0/TK*(-8966.9-2890.51*sqrt(S)-77.942*S+1.726*S**(1.5)-.0993*S**2)+148.0248+137.194*sqrt(S)+
                 1.62247*S+.053105*sqrt(S)*TK+log(TK)*(-24.4344-25.085*sqrt(S)-.2474*S)))



# We are going to have to solve a full system of equations in order to determine the inorganic carbon chemistry.
# The first step will be to solve for the concentration of the H+ ion as a function of conditions, DIC, and ALK.
# This will happen through the use of a fixed point iteration, which apparently works. This is annoying to do in
# python.....


# We begin by defining the function which will iterate. We shall call it 'G':

def G(Hplus,A,C,S,T):
    K1C = K1(S,T)
    K2C = K2(S,T)
    KBC = KB(S,T)
    KWC = KW(S,T)
    c = 11.185
    return -A+C*(1.0/(Hplus**2.0/(K1C*K2C)+1.0+Hplus/K2C)+1.0/(Hplus/K1C+1+K2C/Hplus))+KWC/Hplus+c*S/(1+Hplus/KBC)


# Now we will use an iteration scheme to solve Hplus = G(Hplus)

#def HP(A,C,S,T):
    



# bh and bl are the value of K0 at the surface of the upper and lower lattitude box, respectively.
# Units are in mol/(micro-atm*m^3)

bh = K0(SH,TH)*1e-3
bl = K0(SL,TL)*1e-3


# KS is used to calculate the value of pCO2 from the total carbon concentration of the water as well as from
# the alkalinity. It has units of atm*kg/mol C

def KS(S,T):
    return K2(S,T)/(K0(S,T)*K1(S,T))


# PCO2 is the partial pressure of CO2 in the water. C and A have units of micromol/L, and KS has units of atm*L/mol,
# so PCO2 has units of micro atmospheres. 

def PCO2(A,C,T,S):
    return KS(S,T)*(2*C-A)**2/(A-C)



def CAestimate(A,Hp,S,T):
    c=11.79
    return A-KB(S,T)/(KB(S,T)+Hp)*c*S


def HpEstimate(CA,C,S,T):
    a = CA
    b = K1(S,T)*(CA-C)
    c = K1(S,T)*K2(S,T)*(CA-2*C)
    return (-b+sqrt(b**2-4*a*c))/(2*a)


def HPSolver(A,C,S,T):
    
    counter = 0
    HPold = 1e-2
    CA = CAestimate(A,HPold,S,T)
    Hpnew = HpEstimate(CA,C,S,T)
    while abs(Hpnew-HPold)>1e-6 and counter < 100:
        HPold = Hpnew
        CA = CAestimate(A,Hpnew,S,T)
        
        Hpnew = HpEstimate(CA, C, S, T)
        counter = counter + 1
    return Hpnew



def PCO2new(A,C,S,T):
    
    HP = HPSolver(A,C,S,T)
    return C/( K0(S,T)*(1+K1(S,T)/HP+K1(S,T)*K2(S,T)/HP**2))









def fluxCH(CA, CD, CH, CL, AH,Temp,S, PD, PH, PL):
    if PH<1e-3 and T*PL-(T+fhd)*PH+fhd*PD-phP<0:
        return T*CL-(T+fhd)*CH+fhd*CD+fah/secondsPerDay*bh*(CA-PCO2new(AH,CH,S,Temp))-(T*PL-(T+fhd)*PH+fhd*PD )/rpc
    return T*CL-(T+fhd)*CH+fhd*CD+fah/secondsPerDay*bh*(CA-PCO2new(AH,CH,S,Temp))-ph

def fluxCL(CA, CD, CH, CL, AL, Temp,S,PD):
    return T*CD-(T)*CL+fal/secondsPerDay*bl*(CA-PCO2new(AL,CL,S,Temp))-pl(PD)

def fluxCA(CA,CH,CL,AH,AL,TH,TL,SH,SL):
    return -fal*bl*(CA-PCO2new(AL,CL,SL,TL))/secondsPerDay-fah*bh*(CA-PCO2new(AH,CH,SH,TH))/secondsPerDay


# Now that we have defined all of the individual flux functions, 
# we synthesize them together and define the overall flux function,
# which will in principle be a functio of time

# For checking conservation:

def molCTotal(CA,CD,CH,CL):
    return CA*molesAtmosphere*1e-6+CD*volumeD*1e-3+CH*volumeH*1e-3+CL*volumeL*1e-3



def rightHandSide(t,state):
    CA = state[0]
    CD = state[1]
    CH = state[2]
    CL = state[3]
    AD = state[4]
    AH = state[5]
    AL = state[6]
    PD = state[7]
    PH = state[8]
    
    
    rhsVec = np.zeros((9,1))
    rhsVec[0] = 1e6*fluxCA(CA, CH, CL, AH, AL, TH, TL,SH,SL)/molesAtmosphere
    rhsVec[1] = 1e3*fluxCD(CD, CH, CL,PD,PH,PL)/volumeD
    rhsVec[2] = 1e3*(fluxCH(CA, CD, CH, CL, AH, TH,SH,PD,PH,PL)/volumeH)
    rhsVec[3] = 1e3*(fluxCL(CA, CD, CH, CL, AL, TL,SL,PD)/volumeL)
    rhsVec[4] = 1e3*(fluxAD(AD, AH, AL,PD,PH,PL)/volumeD)
    rhsVec[5] = 1e3*(fluxAH(AD, AH, AL,PD,PH,PL)/volumeH)
    rhsVec[6] = 1e3*(fluxAL(AD, AH, AL,PD)/volumeL)
    rhsVec[7] = 1e3*(fluxPD(PD, PH, PL)/volumeD)
    rhsVec[8] = 1e3*(fluxPH(PD, PH, PL)/volumeH)
    
    return rhsVec


    





In [559]:
#state1=np.array((265,2269.0,2121.0,1941.0,2372.0,2329.0,2290.0,2.15,1.02))
stateIceAgeEq=np.array((200.0,2506.0, 2213.0, 2044.0, 2601.0, 2517.0, 2517.0,2.3,2e-3))
CA = stateIceAgeEq[0]
CD = stateIceAgeEq[1]
CH = stateIceAgeEq[2]
CL = stateIceAgeEq[3]
AD = stateIceAgeEq[4]
AH = stateIceAgeEq[5]
AL = stateIceAgeEq[6]
PD = stateIceAgeEq[7]
PH = stateIceAgeEq[8]
    




In [615]:
# In this box we will try to find the equilibrium solutions:

stateIceAgeEq=np.array((200.0,2406.0, 2113.0, 1900.0, 2401.0, 2417.0, 2417.0,2.2,0))
CA0 = stateIceAgeEq[0]
CD0 = stateIceAgeEq[1]
CH0 = stateIceAgeEq[2]
CL0 = stateIceAgeEq[3]
AD0 = stateIceAgeEq[4]
AH0 = stateIceAgeEq[5]
AL0 = stateIceAgeEq[6]
PD0 = stateIceAgeEq[7]
PH0 = stateIceAgeEq[8]
    

S=34.78


# Constants:

TotalP =volumeH*PH0 + volumeD*PD0 
TotalC = volumeH*CH0 + volumeL*CL0 + volumeD*CD0 + molesAtmosphere*CA0
TotalA = volumeH*AH0 + volumeL*AL0 + volumeD*AD0

subProblem = np.matrix([[volumeH, volumeD, volumeL, 0.0, 0.0, 0.0, 0.0],
                        [0.0, 0.0, 0.0, volumeD, molesAtmosphere, 0.0, 0.0],
                        [0.0, 0.0, 0.0, 0.0, 0.0, volumeH, volumeD],
                        [0.0, 0.0, 0.0, 0.0, 0.0, fhd+T, -fhd],
                        [fhd+T,-fhd, -T, 0.0, 0.0, 0.0, 0.0],
                        [0.0, -T, T, 0.0, 0.0, 0.0, T*rpa],
                        [0.0, 0.0, 0.0, T+fhd, 0.0, 0.0, -T*rpcl]])

invSubProblem = numpy.matrix.getI(subProblem)


def subVector(CH, CL):
    return np.matrix([TotalA, TotalC-volumeH*CH-volumeL*CL, TotalP, -phP, -phP*rpa, 0, phP*rpch+(fhd+T)*CH])



def zeroFunction(vec):
    CL = vec[0]
    CH = vec[1]
    subvec = numpy.matrix.getT(subVector(CH,CL))
    solVec = invSubProblem*subvec
    AH = solVec[0,0]
    AD = solVec[1,0]
    AL = solVec[2,0]
    CD = solVec[3,0]
    CA = solVec[4,0]
    PH = solVec[5,0]
    PD = solVec[6,0]
    
    lowBalance = T*CL+fal/secondsPerDay*bl*(-CA+PCO2new(AL,CL,S,TL))-T*CD+T*PD*rpcl
    highBalance = (T+fhd)*CH+fah/secondsPerDay*bh*(PCO2new(AH,CH,S,TH)-CA)+ph-fhd*CD-T*CL
    return [lowBalance, highBalance]









def equilibrium(initialStateVec):
    CA0 = initialStateVec[0]
    CD0 = initialStateVec[1]
    CH0 = initialStateVec[2]
    CL0 = initialStateVec[3]
    AD0 = initialStateVec[4]
    AH0 = initialStateVec[5]
    AL0 = initialStateVec[6]
    PD0 = initialStateVec[7]
    PH0 = initialStateVec[8]
    

    S=34.78


    # Constants:

    TotalP =volumeH*PH0*1e3 + volumeD*PD0*1e3 
    TotalC = volumeH*CH0*1e3 + volumeL*CL0*1e3 + volumeD*CD0*1e3 + molesAtmosphere*CA0
    TotalA = volumeH*AH0*1e3 + volumeL*AL0*1e3 + volumeD*AD0*1e3
    print TotalC
    subProblem = np.matrix([[volumeH*1e3, volumeD*1e3, volumeL*1e3, 0.0, 0.0, 0.0, 0.0],
                            [0.0, 0.0, 0.0, volumeD*1e3, molesAtmosphere, 0.0, 0.0],
                            [0.0, 0.0, 0.0, 0.0, 0.0, volumeH*1e3, volumeD*1e3],
                            [0.0, 0.0, 0.0, 0.0, 0.0, fhd+T, -fhd],
                            [fhd+T,-fhd, -T, 0.0, 0.0, 0.0, 0.0],
                            [0.0, -T, T, 0.0, 0.0, 0.0, T*rpa],
                            [0.0, 0.0, 0.0, T+fhd, 0.0, 0.0, -T*rpcl]])

    invSubProblem = numpy.matrix.getI(subProblem)



    def subVector(CH, CL):
        return np.matrix([TotalA, TotalC-volumeH*CH-volumeL*CL, TotalP, -phP, -phP*rpa, 0, phP*rpch+(fhd+T)*CH])


    
    #def zeroFunction(vec):
    #    CL = vec[0]
    #    CH = vec[1]
    #    subvec = numpy.matrix.getT(subVector(CH,CL))
    #    solVec = invSubProblem*subvec
    #    AH = solVec[0,0]
    #    AD = solVec[1,0]
    #    AL = solVec[2,0]
    #    CD = solVec[3,0]
    #    CA = solVec[4,0]
    #    PH = solVec[5,0]
    #    PD = solVec[6,0]

    #    lowBalance = T*CL+fal/secondsPerDay*bl*(-CA+PCO2new(AL,CL,S,TL))-T*CD+T*PD*rpcl
    #    highBalance = (T+fhd)*CH+fah/secondsPerDay*bh*(PCO2new(AH,CH,S,TH)-CA)+ph-fhd*CD-T*CL
    #    return [lowBalance, highBalance]

    #solution=scipy.optimize.root(zeroFunction,np.array([2150,2150])).x
    
    CHEq = 2150#solution[0]
    CLEq = 2150#solution[1]
    
    # subvec=numpy.matrix.getT(subVector(CHEq,CLEq))
    subvec=numpy.matrix.getT(subVector(2150,2150))
    subSolVec = invSubProblem*subvec
    AHEq = subSolVec[0,0]
    ADEq = subSolVec[1,0]
    ALEq = subSolVec[2,0]
    CDEq = subSolVec[3,0]
    CAEq = subSolVec[4,0]
    PHEq = subSolVec[5,0]
    PDEq = subSolVec[6,0]
    print volumeH*CHEq*1e3 + volumeL*CLEq*1e3 + volumeD*CDEq*1e3 + molesAtmosphere*CAEq
    return [CAEq, CDEq, CHEq, CLEq,ADEq, AHEq, ALEq, PDEq, PHEq]







In [616]:
equilibrium(stateIceAgeEq)

3.9113055e+23
4.006025685e+23


[362.06193077505486,
 2263.5411177811807,
 2150,
 2150,
 2403.4890446236145,
 2370.5534532062075,
 2320.3596118859914,
 2.187616650990087,
 1.3208905610574346]

In [594]:
equilibrium(equilibrium(vecStart))

[3.1680000000000013e+20, 4.0056335898961106e+23, 3.5640296999999997e+23]
[3.1680000000000026e+20, 4.1008490112872427e+23, 3.5640296999999997e+23]


[539.71217670827446,
 2173.1048392592884,
 2059.5637214781073,
 2232.6983246227651,
 2403.4890446236145,
 2370.5534532062075,
 2320.3596118859914,
 2.1876166509900887,
 1.3208905610574357]

In [425]:
TotalC

5.4359003004300004e+22

In [426]:
300*molesAtmosphere

5.4e+22

In [428]:
volumeD*2000*1e3

2.7936e+23

In [416]:
molesAtmosphere*200*1e-6

3.6e+16

1.185

In [834]:
def CAestimate(A,Hp,S,T):
    c=11.79
    return A-KB(S,T)/(KB(S,T)+Hp)*c*S


def HpEstimate(CA,C,S,T):
    a = CA
    b = K1(S,T)*(CA-C)
    c = K1(S,T)*K2(S,T)*(CA-2*C)
    return (-b+sqrt(b**2-4*a*c))/(2*a)


def HPSolver(A,C,S,T):
    
    counter = 0
    HPold = 1e-2
    CA = CAestimate(A,HPold,S,T)
    Hpnew = HpEstimate(CA,C,S,T)
    while abs(Hpnew-HPold)>1e-6 and counter < 100:
        HPold = Hpnew
        CA = CAestimate(A,Hpnew,S,T)
        
        Hpnew = HpEstimate(CA, C, S, T)
        counter = counter + 1
    return Hpnew



def PCO2new(A,C,S,T):
    
    HP = HPSolver(A,C,S,T)
    return C/( K0(S,T)*(1+K1(S,T)/HP+K1(S,T)*K2(S,T)/HP**2))


In [257]:
rightHandSide(0,stateIceAgeEq)

array([[ -9.49580968e-08],
       [  2.80211913e-06],
       [ -1.39815118e-03],
       [  4.90725499e-04],
       [  1.14745372e-06],
       [ -9.90273661e-05],
       [ -1.03092784e-05],
       [  7.00084380e-10],
       [ -7.46756672e-08]])

In [258]:
ph/rpc+plP(2.305)-(T+fhd)*2.305

-186912.21374046803

In [617]:
(T+fhd)*1.02-fhd

64362000.0

2413.7399467416963

In [588]:

rr = ode(rightHandSide).set_integrator('dopri5')
rr.set_initial_value(vecStart,0)
rr.t
t1 = 3600*24*30000
dt = 3600*24*300
while (rr.successful() and rr.t < t1):
    rr.integrate(rr.t+dt)
    
    
    
    


In [589]:
rr.y

array([  3.27626561e+02,   2.30861416e+03,   2.19507372e+03,
         2.02182656e+03,   2.40348904e+03,   2.37055345e+03,
         2.32035961e+03,   2.18761665e+00,   1.32089056e+00])

In [88]:
PCO2new(2515.9,2137.8,34.7,21.5)

282.63543946499237

In [20]:
state1

array([  2.65000000e+02,   2.47952577e+03,   2.31779381e+03,
         2.12109278e+03,   2.59208247e+03,   2.54509278e+03,
         2.50247423e+03,   2.21649485e+00,   1.05154639e+00])

In [848]:
molCTotal(14.4,1699.0, 1549.0, 1412.7)

2.53662012e+17

In [847]:
molCTotal(265,2269.0,2121.0,1941.0)

3.8323881e+17

In [898]:
def fluxCHT(CA, CD, CH, CL, AH,TH,S):
    print fah/secondsPerDay*bh*(CA-PCO2new(AH,CH,TH,S))
    print ph
    print fhd*CD
    print T*CL
    print T*CD
    print (T+fhd)*CH
    return T*CL-(T+fhd)*CH+fhd*CD+fah/secondsPerDay*bh*(CA-PCO2new(AH,CH,TH,S))-ph

def fluxCLT(CA, CD, CH, CL, AL, TL,S,PD):
    print PCO2(AL,CL,TL,S)
    print fal/secondsPerDay*bl*(CA-PCO2new(AL,CL,TL,S))
    print pl(PD)
    print T*CD
    print (T)*CL
    return T*CD-(T)*CL+fal/secondsPerDay*bl*(CA-PCO2new(AL,CL,TL,S))-pl(PD)

def fluxCAT(CA,CH,CL,AH,AL,TH,TL,SH,SL):
    print fal*bl*(CA-PCO2new(AL,CL,TL,SL))/secondsPerDay 
    print fah*bh*(CA-PCO2new(AH,CH,TH,SH))/secondsPerDay
    return -fal*bl*(CA-PCO2new(AL,CL,TL,SL))/secondsPerDay-fah*bh*(CA-PCO2new(AH,CH,TH,SH))/secondsPerDay


def fluxCDT(CD,CH,CL,PD):
    print pl(PD)
    print ph
    print (T+fhd)
    print CH
    print (T+fhd)*CD
    return (T+fhd)*CH-(T+fhd)*CD+ph+pl(PD)






In [899]:
fluxCHT(265,2269.0,2121.0,1941.0,2329.0,2.0,35.0)+fluxCAT(265.0,2121.0,1941,2329.0,2290.0,2.0,20.0,35.0,35.0)+fluxCLT(265.0,2269.0,2121.0,1941,2290.0,20.0,35.0,2.15)+fluxCDT(2269.0,2121.0,1941.0,2.15)

-22750364.0705
2318000000.0
86448900000.0
49301400000.0
57632600000.0
1.346835e+11
-14638117.9694
-22750364.0705
156.300452765
-14638117.9694
7153910000.0
57632600000.0
49301400000.0
7153910000.0
2318000000.0
63500000.0
2121.0
1.440815e+11


7.152557373046875e-07

In [907]:
fluxCAT(CA,CH,CL, AH, AL, TH, TL,SH,SL)-fluxCA(CA, CH, CL, AH, AL, TH, TL,SH,SL)

-19245807.4489
-23173483.4931


102550767.45626786

In [908]:
fluxCLT(CA, CD, CH, CL, AL, TL,SL,PD)-fluxCL(CA, CD, CH, CL, AL, TL,SL,PD)


162.263607167
-19245807.4489
7153910000.0
57632600000.0
49301400000.0


0.0

In [909]:
fluxCDT(2269.0,2121.0,1941.0,2.15)-fluxCD(CD, CH, CL,PD)

7153910000.0
2318000000.0
63500000.0
2121.0
1.440815e+11


0.0

In [910]:
fluxCH(CA, CD, CH, CL, AH, TH,SH)-fluxCHT(CA, CD, CH, CL, AH, TH,SH)

-23173483.4931
2318000000.0
86448900000.0
49301400000.0
57632600000.0
134683500000.0


0.0

In [188]:
fluxCA(CA, CH, CL, AH, AL, TH, TL,SH,SL)+fluxCD(CD, CH, CL,PD)+fluxCH(CA, CD, CH, CL, AH, TH,SH)+fluxCL(CA, CD, CH, CL, AL, TL,SL,PD)


776.97445075007545

In [171]:
K0(35,20)

2.0461159389354723e-06

In [172]:
KS(35,20)

342.31068220818668

In [275]:
T*2*136*1e-3

2.9908225108225106

ph

In [273]:
ph

2310000.0

 11.48*1e6-2.24*(38.1e6)*1e3

In [501]:
 11.1815*1e6/131-2.24*1e-3*(38.1e6)

10.961832061060704

In [491]:
 10.48*1e6/131

87633.58778625954

In [548]:
state=np.array([200,2506.0, 2213.0, 2044.0, 2601.0, 2517.0, 2517.9, 2.24, 0])

In [549]:
rightHandSide(0,state)*3600*24*365

array([[ -1.58694961e+01],
       [  4.66447423e+00],
       [ -3.64860162e+04],
       [  1.68396381e+04],
       [ -4.88311368e+02],
       [  4.23288090e+04],
       [  4.30488393e+03],
       [ -1.10301409e-04],
       [  1.17654836e-02]])

In [525]:
fluxPH(2.24,.0,.0)

488.5496183335781

In [524]:
fluxPD(2.24,0,0)

-488.5496183335781

In [552]:
bh*fah*(185-200)+bl*fal*(205-200)

568614490.76272392

In [553]:
bl*fal*(205-200)/(bh*fah*(185-200))

-1.0405374655961683

In [537]:
fah

92055000000000.0

In [542]:
fah/fal

0.1764705882352941

In [557]:
T*2506.0-pl(2.24)-ph

8733040000.0

In [575]:
(ph+pl(2.24)+(T+fhd)*2213)/(T*2506+fhd*2506.0)

1.000175036388252

In [559]:
T

9000000.0

In [605]:
(bl*fal*(205-200)+T*2506.0)/(T*2044+pl(2.24))

1.7659159173403836

In [606]:
bl

3.1710395994683516e-05

In [570]:
10/(1+1)

5

In [579]:
ph/pl(2.24)

4.233309099721312

In [582]:
bl

3.1710395994683516e-05

In [583]:
bh

5.7563919260039946e-05

In [587]:
exp(-log(10)*K0(35,0))

0.99994247884458221

In [590]:
exp(-log(10)*KS(35,0.0))

0.98098827130428135

In [589]:
exp(-log(10)*K1(35.0,0.0))

0.99999820560850339

In [597]:
K2(35.0,0.0)

4.1082682313974849e-10

In [594]:
10**(-9.384)

4.1304750199016105e-10

In [598]:
K1(35.0,0.0)

7.7929502451549943e-07

In [599]:
10**(-6.106)

7.83429642766212e-07

In [600]:
K0(35.0,0.0)

0.063239662416791659

In [601]:
10**(-1.202)

0.0628058358813318

In [602]:
KS(35.0,0.0)

0.0083361850138135782

In [604]:
10**(-2.076)

0.008394599865193973

In [609]:
bl*1e3

0.031710395994683518

In [610]:
K0(35.0,20.0)

0.032542518039698362

In [611]:
PCO2(2517.0,2044.0,20.0,35.0)

112.30284682574637

In [612]:
PCO2(2517.0,2213.0,2.5,34.7)

112.78138406882722

In [616]:
(2*2044-2517)**2/(2517-2044)*KS(35.0,20.0)

112.28464569426991

In [627]:
KS(34.7,20.0)*(2*2012.0-2322.0)**2/(2322.0-2012.0)

199.9002069338033

In [633]:
PCO2(2291.0,2119.0,3.7,34.05)-17/K0(34.05,3.7)

-90.368274044137735

In [629]:
PCO2(2315.0,2003.0, 23.0,34.05)

220.27103538685702

In [630]:
PCO2(2257.0,2049.0,4.7, 32.90)

164.80974934924421

In [631]:
PCO2(2308.0,2026.0,18.1,34.75)

212.42404364598323

In [636]:
13/K0(32.15,4.70)

241.26386085042466

In [637]:
11/K0(34.75,18.1)

319.91602479685633

In [640]:
28/K0(34.73,3.6)

506.83142137025305

In [644]:
PCO2(2457.0,2073.0,11.0,34.78)

105.86433424373149

In [643]:
7.312/K0(34.78,11.0)

171.13202530437377

In [645]:
KS(34.78,11.0)*1797**2/268.8

171.19397882075489

In [646]:
2457-2073

384

In [647]:
2*2073-2457

1689